<a href="https://colab.research.google.com/github/karellen-kim/training-feature-store/blob/main/hopsworks_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U hopsworks --quiet

In [16]:
import pandas as pd
import hopsworks

In [74]:
project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/500884
Connected. Call `.close()` to terminate connection gracefully.


In [75]:
movie_rating_data = pd.read_csv('/content/hopsworks/ratings_small.csv')
movie_rating_data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [76]:
movie_ratings = fs.get_or_create_feature_group(
    name="movie_ratings",
    version=2,
    description="movie ratings",
    primary_key=["movieId"],
    event_time="timestamp",
    online_enabled=False,
)
movie_ratings.insert(movie_rating_data)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/500884/fs/496707/fg/552285


Uploading Dataframe: 0.00% |          | Rows 0/100004 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: movie_ratings_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/500884/jobs/named/movie_ratings_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7d2911e80070>, None)

In [59]:
movie_data = pd.read_csv('/content/hopsworks/movies.csv')
movie_data = movie_data.dropna(subset=['title','vote_average', 'vote_count'])
movie_data.head()

,id,title,vote_average,vote_count
0,862,Toy Story,7.7,5415.0
1,8844,Jumanji,6.9,2413.0
2,15602,Grumpier Old Men,6.5,92.0
3,31357,Waiting to Exhale,6.1,34.0
4,11862,Father of the Bride Part II,5.7,173.0


In [61]:
movies = fs.get_or_create_feature_group(
    name="movies",
    version=1,
    description="movies",
    primary_key=["id"],
    online_enabled=True,
)
movies.insert(movie_data)

Uploading Dataframe: 0.00% |          | Rows 0/45460 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: movies_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/500884/jobs/named/movies_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7d29302ae740>, None)

In [96]:
selected_features = movies.select(["id", "title"]).join(movie_ratings.select(["movieId", "rating"]), left_on=["id"], right_on=["movieId"])
selected_features.show(5)

Finished: Reading data from Hopsworks, using ArrowFlight (2.48s) 


,id,title,movieid,rating
0,924,Dawn of the Dead,924,5.0
1,165,Back to the Future Part II,165,5.0
2,955,Mission: Impossible II,955,2.5
3,913,The Thomas Crown Affair,913,5.0
4,59118,The Prey,59118,4.5


In [98]:
feature_view = fs.get_or_create_feature_view(
    name='movie_ratings',
    version=1,
    query=selected_features
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/500884/fs/496707/fv/movie_ratings/version/1


In [104]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    description='training dataset',
    test_size=0.2
)

Finished: Reading data from Hopsworks, using ArrowFlight (2.91s) 


In [105]:
X_train.head()

,id,title,movieid,rating
1,165,Back to the Future Part II,165,5.0
2,955,Mission: Impossible II,955,2.5
3,913,The Thomas Crown Affair,913,5.0
4,59118,The Prey,59118,4.5
5,1391,Y Tu Mamá También,1391,1.0
